## Распаковка архива

In [ ]:
!unzip '/content/caltech-101.zip'
!tar -xvzf '/content/caltech-101/101_ObjectCategories.tar.gz'
!tar -xvf '/content/caltech-101/Annotations.tar'
!mkdir '/content/caltech101'
!cp -r '/content/101_ObjectCategories/' '/content/caltech101'
!cp -r '/content/Annotations/' '/content/caltech101'

## Импортирование необходимых библиотек и функций

In [ ]:
import numpy as np
import scipy.sparse as sp
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score
from sklearn.preprocessing import StandardScaler
import networkx as nx
from collections import defaultdict
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import rbf_kernel, pairwise_distances
import torch
from torch import nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.utils.validation import check_array
from sklearn.decomposition import PCA

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

import torch.nn.functional as F

from scipy.sparse.csgraph import laplacian
from scipy.linalg import eigh
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

import random

from sklearn.preprocessing import LabelEncoder

## Дополнительные методы

In [ ]:
def create_constraints(labels, labeled_indices):
    must_link = []
    cannot_link = []
    known_labels = {}
    for idx in labeled_indices:
        known_labels[idx] = labels[idx]

    for i in labeled_indices:
        for j in labeled_indices:
            if i < j:
                if labels[i] == labels[j]:
                    must_link.append((i, j))
                else:
                    cannot_link.append((i, j))
    return must_link, cannot_link, known_labels


def evaluate_clustering(labels_true, labels_pred):
    ari = adjusted_rand_score(labels_true, labels_pred)
    nmi = normalized_mutual_info_score(labels_true, labels_pred)
    fm = fowlkes_mallows_score(labels_true, labels_pred)
    return ari, nmi, fm


def load_caltech101(data_path='/content/dataset/',
                    subset=None,
                    download=True,
                    n_samples=2000):

    transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.Lambda(lambda x: x.convert('RGB')),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    caltech101_dataset = datasets.Caltech101(root=data_path,
                                             download=False,
                                             transform=transform,
                                             target_transform=transforms.Lambda(lambda y: torch.tensor(y, dtype=torch.long)))

    total_samples = len(caltech101_dataset)
    n_samples = min(n_samples, total_samples)
    indices = random.sample(range(total_samples), n_samples)
    from torchvision.models import resnet18
    model = resnet18(pretrained=True).eval()
    data = []
    labels = []
    for idx in indices:
        img, target = caltech101_dataset[idx]
        features = model(torch.tensor(img[np.newaxis, :])).detach().numpy()
        data.append(features.flatten())
        labels.append(target.item())

    data = np.array(data)
    labels = np.array(labels)
    return data, labels

## KMeans

KMeans с прямым учетом ограничений

In [ ]:
class ConstrainedKMeans:
    def __init__(self, n_clusters, must_link=[], cannot_link=[], init='k-means++', n_init=10, max_iter=1000, random_state=None):
        self.n_clusters = n_clusters
        self.must_link = must_link
        self.cannot_link = cannot_link
        self.init = init
        self.n_init = n_init
        self.max_iter = max_iter
        self.random_state = random_state
        self.cluster_centers_ = None
        self.labels_ = None
        self.build_cannot_link_dict()

    def build_must_link_components(self, n_samples):
        if not self.must_link:
            return np.arange(n_samples)
        rows, cols = zip(*self.must_link)
        data = np.ones(len(rows), dtype=int)
        graph = csr_matrix((data, (rows, cols)), shape=(n_samples, n_samples))
        graph = graph + graph.T + csr_matrix(np.eye(n_samples))
        _, labels = connected_components(graph)
        return labels

    def build_cannot_link_dict(self):
        self.cannot_link_dict = defaultdict(set)
        for i, j in self.cannot_link:
            self.cannot_link_dict[i].add(j)
            self.cannot_link_dict[j].add(i)

    def fit(self, X):
        n_samples, n_features = X.shape
        must_link_components = self.build_must_link_components(n_samples)
        unique_components = np.unique(must_link_components)
        n_components = len(unique_components)
        component_to_indices = defaultdict(list)
        for idx, comp in enumerate(must_link_components):
            component_to_indices[comp].append(idx)
        kmeans = KMeans(n_clusters=self.n_clusters, init=self.init, n_init=self.n_init, max_iter=1, random_state=self.random_state)
        kmeans.fit(X)
        initial_centroids = kmeans.cluster_centers_
        component_to_cluster = {}
        for comp in unique_components:
            indices = component_to_indices[comp]
            comp_points = X[indices]
            mean_point = np.mean(comp_points, axis=0)
            closest_cluster = np.argmin(np.linalg.norm(initial_centroids - mean_point, axis=1))
            component_to_cluster[comp] = closest_cluster
        self.labels_ = np.zeros(n_samples, dtype=int)
        for comp, indices in component_to_indices.items():
            self.labels_[indices] = component_to_cluster[comp]
        self.cluster_centers_ = initial_centroids.copy()

        for _ in range(self.max_iter):
            centroids = np.zeros((self.n_clusters, n_features))
            counts = np.bincount(self.labels_, minlength=self.n_clusters)

            for k in range(self.n_clusters):
                if counts[k] > 0:
                    centroids[k] = X[self.labels_ == k].mean(axis=0)
            distances_cache = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
            new_labels = self.labels_.copy()
            changed = False

            for comp, indices in component_to_indices.items():
                current_cluster = new_labels[indices[0]]
                has_conflict = False
                for idx in indices:
                    for other in self.cannot_link_dict[idx]:
                        if new_labels[other] == current_cluster:
                            has_conflict = True
                            break
                    if has_conflict:
                        break

                if not has_conflict:
                    continue
                comp_mean = np.mean(X[indices], axis=0)
                for new_cluster in np.argsort(np.linalg.norm(centroids - comp_mean, axis=1)):
                    if new_cluster == current_cluster:
                        continue
                    valid = True
                    for idx in indices:
                        for other in self.cannot_link_dict[idx]:
                            if new_labels[other] == new_cluster:
                                valid = False
                                break
                        if not valid:
                            break
                    if valid:
                        for idx in indices:
                            new_labels[idx] = new_cluster
                        changed = True
                        break
            self.labels_ = new_labels
            self.cluster_centers_ = centroids
            if not changed:
                break

        return self

    def predict(self, X):
        return np.argmin(np.linalg.norm(X[:, np.newaxis] - self.cluster_centers_, axis=2), axis=1)

KMeans с регуляризацией

In [ ]:
class RegularizedKMeans(nn.Module):
    def __init__(self, n_clusters, n_features, must_link=[], cannot_link=[], alpha=1.0, beta=1.0):
        super(RegularizedKMeans, self).__init__()
        self.n_clusters = n_clusters
        self.n_features = n_features
        self.must_link = must_link
        self.cannot_link = cannot_link
        self.alpha = alpha
        self.beta = beta
        torch.manual_seed(42)
        self.cluster_centers = nn.Parameter(torch.randn(n_clusters, n_features))
        self.loss_fn = nn.MSELoss(reduction='mean')

    def forward(self, X):
        distances = torch.cdist(X, self.cluster_centers, p=2)
        cluster_assignments = torch.argmin(distances, dim=1)
        clustering_loss = 0.0
        for i in range(self.n_clusters):
            mask = (cluster_assignments == i)
            if mask.any():
                cluster_points = X[mask]
                clustering_loss += self.loss_fn(cluster_points, self.cluster_centers[i].expand_as(cluster_points))
        clustering_loss /= self.n_clusters
        reg_loss = self.regularization_loss(distances)
        total_loss = clustering_loss + self.alpha * reg_loss
        return cluster_assignments, total_loss

    def regularization_loss(self, distances):
        batch_size = distances.size(0)
        weights = F.softmax(-self.beta * distances, dim=1)
        reg_loss = 0.0
        must_count = 0
        cannot_count = 0

        for i, j in self.must_link:
            if i < batch_size and j < batch_size:
                p_i = weights[i] + 1e-8
                p_j = weights[j] + 1e-8
                kl_loss = F.kl_div(p_i.log(), p_j, reduction='sum') + F.kl_div(p_j.log(), p_i, reduction='sum')
                reg_loss += kl_loss
                must_count += 1

        for i, j in self.cannot_link:
            if i < batch_size and j < batch_size:
                dot_product = torch.dot(weights[i], weights[j])
                reg_loss += dot_product
                cannot_count += 1
        total_constraints = must_count + cannot_count
        if total_constraints > 0:
            reg_loss /= total_constraints

        return reg_loss

    def fit(self, X, epochs=100, lr=0.01):
        X_tensor = torch.from_numpy(X).float()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            _, loss = self.forward(X_tensor)
            loss.backward()
            optimizer.step()

    def predict(self, X):
        X_tensor = torch.from_numpy(X).float()
        assignments, _ = self.forward(X_tensor)
        return assignments.numpy()

KMeans с инициализацией центроидов

In [ ]:
class InitializedKMeans(KMeans):
    def __init__(self, n_clusters, init_centroids=None,
                 n_init=5, max_iter=300, random_state=None):
        self.init_centroids = init_centroids
        self.n_provided = len(init_centroids) if init_centroids is not None else 0
        if self.n_provided > 0:
            init_centroids = np.array(init_centroids)
            n_missing = n_clusters - self.n_provided
            rng = np.random.RandomState(random_state)

            if n_missing > 0:
                feature_dim = init_centroids.shape[1]
                random_centroids = rng.normal(loc=np.mean(init_centroids, axis=0),
                                              scale=np.std(init_centroids, axis=0) + 1e-6,
                                              size=(n_missing, feature_dim))
                init = np.vstack([init_centroids, random_centroids])
            else:
                init = init_centroids
        else:
            init = 'k-means++'

        super().__init__(n_clusters=n_clusters,
                        init=init,
                        n_init=n_init,
                        max_iter=max_iter,
                        random_state=random_state)

    def fit(self, X, y=None, sample_weight=None):
        super().fit(X, y=y, sample_weight=sample_weight)
        if self.n_provided > 0:
            self.cluster_centers_[:self.n_provided] = np.array(self.init_centroids)
        return self

## Графовые подходы

Графовый иерархический подход

In [ ]:
class ConstrainedGraphClustering:
    def __init__(self, n_clusters, must_link=[], cannot_link=[],
                 affinity='rbf', gamma=0.1, n_neighbors=5, random_state=None):
        self.n_clusters = n_clusters
        self.must_link = must_link
        self.cannot_link = cannot_link
        self.affinity = affinity
        self.gamma = gamma
        self.n_neighbors = n_neighbors
        self.random_state = random_state
        self.labels_ = None

    def fit_predict(self, X):
        if self.gamma is None and self.affinity == 'rbf':
            self.gamma = 1.0 / (X.shape[1] * X.var()) if X.shape[1] > 0 else 0.1
        sim_matrix = self.build_similarity_with_constraints(X)
        distance_matrix = 1 - sim_matrix
        np.fill_diagonal(distance_matrix, 0)
        model = AgglomerativeClustering(n_clusters=self.n_clusters,
                                        metric='precomputed',
                                        linkage='average')
        self.labels_ = model.fit_predict(distance_matrix)
        return self.labels_

    def build_similarity_with_constraints(self, X):
        n = X.shape[0]
        if self.affinity == 'rbf':
            sim = rbf_kernel(X, gamma=self.gamma)
        elif self.affinity == 'cosine':
            sim = cosine_similarity(X)
        elif self.affinity == 'nearest_neighbors':
            sim = np.zeros((n, n))
            distances = pairwise_distances(X)
            for i in range(n):
                neighbors = np.argsort(distances[i])[1:self.n_neighbors + 1]
                sim[i, neighbors] = 1
                sim[neighbors, i] = 1

        for i, j in self.must_link:
            sim[i, j] = sim[j, i] = 1.0
        for i, j in self.cannot_link:
            sim[i, j] = sim[j, i] = 0.0
        return sim

Графовый подход с регуляризацией

In [ ]:
class RegularizedGraphClustering(nn.Module):
    def __init__(self, n_clusters, n_nodes, must_link=[], cannot_link=[], alpha=1.0, margin=1.0):
        super().__init__()
        self.n_clusters = n_clusters
        self.n_nodes = n_nodes
        self.alpha = alpha
        self.margin = margin
        self.cluster_probs = nn.Parameter(torch.randn(n_nodes, n_clusters))
        nn.init.xavier_uniform_(self.cluster_probs)
        self.register_constraints(must_link, cannot_link)

    def register_constraints(self, must_link, cannot_link):
        if must_link:
            self.register_buffer('ml_i', torch.tensor([i for i, _ in must_link], dtype=torch.long))
            self.register_buffer('ml_j', torch.tensor([j for _, j in must_link], dtype=torch.long))
        else:
            self.ml_i = self.ml_j = None
        if cannot_link:
            self.register_buffer('cl_i', torch.tensor([i for i, _ in cannot_link], dtype=torch.long))
            self.register_buffer('cl_j', torch.tensor([j for _, j in cannot_link], dtype=torch.long))
        else:
            self.cl_i = self.cl_j = None

    def constraint_loss(self, P):
        loss = 0.0
        if self.ml_i is not None:
            diff = P[self.ml_i] - P[self.ml_j]
            loss += torch.mean(torch.sum(diff**2, dim=1))
        if self.cl_i is not None:
            diff = P[self.cl_i] - P[self.cl_j]
            dist = torch.sum(diff**2, dim=1)
            loss += torch.mean(torch.clamp(self.margin - dist, min=0.0))

        return loss

    def forward(self, adj_matrix):
        P = torch.softmax(self.cluster_probs, dim=1)
        cluster_assignment = torch.argmax(P, dim=1)
        graph_loss = self.graph_cut_loss(P, adj_matrix)
        constraint_loss = self.constraint_loss(P)
        total_loss = graph_loss + self.alpha * constraint_loss
        return cluster_assignment, total_loss

    def graph_cut_loss(self, P, adj_matrix):
        intra_cluster = torch.sum(adj_matrix * (P @ P.T))
        total_connections = torch.sum(adj_matrix)
        cut_loss = 1 - (intra_cluster / total_connections)
        return cut_loss

    def fit(self, adj_matrix, epochs=100, lr=0.01, verbose=False):
        adj_matrix = torch.FloatTensor(adj_matrix)
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            _, loss = self.forward(adj_matrix)
            loss.backward()
            optimizer.step()

            if verbose and epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    def predict(self):
        with torch.no_grad():
            P = torch.softmax(self.cluster_probs, dim=1)
            return torch.argmax(P, dim=1).numpy()

Графовый подход с распространением меток

In [ ]:
class LabelPropagationClustering:
    def __init__(self, n_clusters, known_labels, n_neighbors=7, max_iter=30, tol=1e-3):
        self.n_clusters = n_clusters
        self.known_labels = known_labels
        self.n_neighbors = n_neighbors
        self.max_iter = max_iter
        self.tol = tol
        self.labels_ = None

    def fit_predict(self, X):
        n_samples = X.shape[0]
        adj_matrix = kneighbors_graph(X,
                                      self.n_neighbors,
                                      mode='connectivity',
                                      include_self=True)
        adj_matrix = adj_matrix.maximum(adj_matrix.T)
        labels = -np.ones(n_samples, dtype=int)
        for idx, lbl in self.known_labels.items():
            if idx < n_samples:
                labels[idx] = lbl
        label_probs = np.zeros((n_samples, self.n_clusters))
        for i in range(n_samples):
            if labels[i] != -1:
                label_probs[i, labels[i]] = 1.0
        for _ in range(self.max_iter):
            prev_probs = label_probs.copy()
            new_probs = np.zeros_like(label_probs)
            for i in range(n_samples):
                if labels[i] == -1:
                    _, neighbors = adj_matrix[i].nonzero()

                    if neighbors.size > 0:
                        new_probs[i] = np.mean(prev_probs[neighbors], axis=0)
                    else:
                        new_probs[i] = prev_probs[i]
                else:
                    new_probs[i] = prev_probs[i]
            if np.abs(new_probs - prev_probs).sum() < self.tol:
                break

            label_probs = new_probs

        self.labels_ = np.argmax(label_probs, axis=1)
        return self.labels_

## Спектральные подходы

Спектральная кластеризация с прямым изменением матрицы близости

In [ ]:
class ConstrainedSpectralClustering:
    def __init__(self, n_clusters, must_link=[], cannot_link=[],
                affinity='rbf', gamma=0.1, n_neighbors=7, random_state=None):

        self.n_clusters = n_clusters
        self.must_link = must_link
        self.cannot_link = cannot_link
        self.affinity = 'nearest_neighbors'
        self.gamma = gamma
        self.n_neighbors = n_neighbors
        self.random_state = random_state
        self.labels_ = None

    def fit_predict(self, X):
        similarity_matrix = self.build_similarity_graph(X)

        for i, j in self.must_link:
            similarity_matrix[i, j] = 1.0
            similarity_matrix[j, i] = 1.0
        for i, j in self.cannot_link:
            similarity_matrix[i, j] = 0.0
            similarity_matrix[j, i] = 0.0

        spectral = SpectralClustering(n_clusters=self.n_clusters,
                                      affinity='precomputed',
                                      random_state=self.random_state,
                                      assign_labels='kmeans')

        self.labels_ = spectral.fit_predict(similarity_matrix)
        return self.labels_

    def build_similarity_graph(self, X):
        if self.affinity == 'rbf':
            return rbf_kernel(X, gamma=0.1)
        elif self.affinity == 'cosine':
            return cosine_similarity(X)
        elif self.affinity == 'nearest_neighbors':
            k = 7
            adjacency = np.zeros((X.shape[0], X.shape[0]))
            distances = pairwise_distances(X)
            for i in range(X.shape[0]):
                nearest_indices = np.argsort(distances[i])[1:k+1]
                adjacency[i, nearest_indices] = 1
                adjacency[nearest_indices, i] = 1
            return adjacency

Спектральная кластеризация с регуляризацией

In [ ]:
class RegularizedSpectralClustering(nn.Module):
    def __init__(self, n_clusters, must_link=[], cannot_link=[], alpha=1.0, margin=1.0):
        super(RegularizedSpectralClustering, self).__init__()
        self.n_clusters = n_clusters
        self.must_link = must_link
        self.cannot_link = cannot_link
        self.alpha = alpha
        self.margin = margin
        torch.manual_seed(42)
        #self.embeddings = nn.Parameter(torch.randn(n_nodes, n_clusters))
        self.embeddings = None

    def spectral_init(self, similarity_matrix, n_components):
        L = laplacian(similarity_matrix, normed=True)
        eigvals, eigvecs = eigh(L)
        X_spec = torch.from_numpy(eigvecs[:, :n_components]).float()
        self.embeddings = nn.Parameter(X_spec)

    def forward(self):
        soft_assignments = torch.softmax(self.embeddings, dim=1)
        cluster_assignments = torch.argmax(soft_assignments, dim=1)
        cluster_loss = -torch.mean(torch.sum(soft_assignments * torch.log(soft_assignments + 1e-10), dim=1))
        reg_loss = self.constraint_loss(soft_assignments)
        total_loss = cluster_loss + self.alpha * reg_loss
        return cluster_assignments, total_loss

    def constraint_loss(self, soft_assignments):
        loss = 0.0
        for i, j in self.must_link:
            loss += torch.sum((soft_assignments[i] - soft_assignments[j]) ** 2)
        for i, j in self.cannot_link:
            dist = torch.sum((soft_assignments[i] - soft_assignments[j]) ** 2)
            loss += torch.clamp(self.margin - dist, min=0.0)
        return loss

    def fit(self, similarity_matrix, epochs=100, lr=0.01):
        self.spectral_init(similarity_matrix, n_components=self.n_clusters)
        optimizer = optim.Adam([self.embeddings], lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            _, loss = self.forward()
            loss.backward()
            optimizer.step()

    def predict(self):
        with torch.no_grad():
            soft_assignments = torch.softmax(self.embeddings, dim=1)
            return torch.argmax(soft_assignments, dim=1).numpy()

Спектральная регуляризация с воздействием матрицей ограничений на матрицу близости

In [ ]:
class ModifiedSimilaritySpectralClustering:
    def __init__(self, n_clusters, must_link=[], cannot_link=[],
                affinity='rbf', gamma=0.1, constraint_strength=0.5,
                random_state=None):
        self.n_clusters = n_clusters
        self.must_link = must_link
        self.cannot_link = cannot_link
        self.affinity = 'nearest_neighbors'
        self.gamma = gamma
        self.constraint_strength = constraint_strength
        self.random_state = random_state
        self.labels_ = None

    def fit_predict(self, X):
        X = check_array(X)
        n_samples = X.shape[0]
        similarity = self.build_similarity_graph(X)
        constraint = np.zeros_like(similarity)
        for i, j in self.must_link:
            if i < n_samples and j < n_samples:
                constraint[i,j] = self.constraint_strength
                constraint[j,i] = self.constraint_strength

        for i, j in self.cannot_link:
            if i < n_samples and j < n_samples:
                constraint[i,j] = -self.constraint_strength
                constraint[j,i] = -self.constraint_strength
        modified = similarity + constraint
        modified = np.clip(modified, 1e-8, None)
        model = SpectralClustering(n_clusters=self.n_clusters,
                                  affinity='precomputed',
                                  random_state=self.random_state,)
                                  #n_components=min(self.n_clusters*2, n_samples-1)
        self.labels_ = model.fit_predict(modified)
        return self.labels_

    def build_similarity_graph(self, X):
        if self.affinity == 'rbf':
            return rbf_kernel(X, gamma=0.1)
        elif self.affinity == 'cosine':
            return cosine_similarity(X)
        elif self.affinity == 'nearest_neighbors':
            k = 7
            adjacency = np.zeros((X.shape[0], X.shape[0]))
            distances = pairwise_distances(X)
            for i in range(X.shape[0]):
                nearest_indices = np.argsort(distances[i])[1:k+1]
                adjacency[i, nearest_indices] = 1
                adjacency[nearest_indices, i] = 1
            return adjacency

## Запуск экспериментов

In [ ]:
def compare_algorithms(X, y, n_clusters, labeled_percentages=[0, 0.005, 0.01, 0.02, 0.05, 0.1], random_state=42):
    results = {}
    pca = PCA(n_components=50)
    X_reduced = pca.fit_transform(X)
    X_train = X_reduced
    y_train = y

    for labeled_percent in labeled_percentages:
        results[labeled_percent] = {}
        print(f"\nПроцент размеченных данных: {labeled_percent * 100}%")

        n_labeled = int(labeled_percent * len(y_train))
        labeled_indices = np.random.choice(len(y_train), n_labeled, replace=False)
        must_link1, cannot_link1, known_labels = create_constraints(y_train, labeled_indices)
        for i in range (3):
          must_link = must_link1.copy()
          cannot_link = cannot_link1.copy()
          if i == 1:
            must_link = []
          if i == 2:
            cannot_link = []
          print("Количество must-link: ", len(must_link))
          print("Количество cannot-link: ", len(cannot_link))
          # 1. K-Means
          # a) Прямая проверка ограничений
          ckmeans = ConstrainedKMeans(n_clusters=n_clusters, must_link=must_link, cannot_link=cannot_link, random_state=random_state)
          ckmeans.fit(X_train)
          y_pred_ckmeans = ckmeans.predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_ckmeans)
          results[labeled_percent]['ConstrainedKMeans'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  K-Means (Прямая проверка): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          # б) K-Means с регуляризацией
          regularized_kmeans = RegularizedKMeans(n_clusters=n_clusters, n_features=X_train.shape[1], must_link=must_link, cannot_link=cannot_link)
          regularized_kmeans.fit(X_train, epochs=300, lr=0.01)
          y_pred_reg_kmeans = regularized_kmeans.predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_reg_kmeans)
          results[labeled_percent]['RegularizedKMeans'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  K-Means (Регуляризация): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          # в) K-Means с инициализацией на основе известных меток
          X_labeled = X_train[labeled_indices]
          known_labels_dict = {i:y_train[i] for i in labeled_indices}
          cluster_points = defaultdict(list)

          for idx in labeled_indices:
              label = y_train[idx]
              cluster_points[label].append(X_train[idx])

          init_centroids = []
          for label, points in sorted(cluster_points.items()):
              init_centroids.append(np.mean(points, axis=0))

          initialized_kmeans = InitializedKMeans(
              n_clusters=n_clusters,
              init_centroids=init_centroids,
              random_state=random_state)

          initialized_kmeans.fit(X_train)
          y_pred_init_kmeans = initialized_kmeans.predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_init_kmeans)
          results[labeled_percent]['InitializedKMeans'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  K-Means (Инициализация): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          # 2. Графовые алгоритмы
          # a) Учет ограничений в весах ребер
          constrained_graph_clustering = ConstrainedGraphClustering(n_clusters=n_clusters, must_link=must_link, cannot_link=cannot_link, random_state=random_state)
          y_pred_graph = constrained_graph_clustering.fit_predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_graph)
          results[labeled_percent]['ConstrainedGraphClustering'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  Графовый (Веса ребер): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          #б) Графовый с регуляризацией
          similarity_matrix = constrained_graph_clustering.build_similarity_with_constraints(X_train)
          regularized_graph_clustering = RegularizedGraphClustering(n_clusters=n_clusters, n_nodes = X_train.shape[0], must_link=must_link, cannot_link=cannot_link)
          regularized_graph_clustering.fit(similarity_matrix, epochs=300, lr=0.01)
          y_pred_reg_graph = regularized_graph_clustering.predict()
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_reg_graph)
          results[labeled_percent]['RegularizedGraphClustering'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  Графовый (Регуляризация): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          # в) Распространение меток
          label_propagation = LabelPropagationClustering(n_clusters=n_clusters, known_labels=known_labels)
          y_pred_label_prop = label_propagation.fit_predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_label_prop)
          results[labeled_percent]['LabelPropagation'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  Распространение меток: ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          #3. Спектральная кластеризация
          #a) Учет ограничений в матрице схожести
          constrained_spectral_clustering = ConstrainedSpectralClustering(n_clusters=n_clusters, must_link=must_link, cannot_link=cannot_link, random_state=random_state)
          y_pred_spectral = constrained_spectral_clustering.fit_predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_spectral)
          results[labeled_percent]['ConstrainedSpectralClustering'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  Спектральная (Матрица схожести): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          # б) Спектральная кластеризация с регуляризацией
          similarity_matrix_spectral = constrained_spectral_clustering.build_similarity_graph(X_train)
          regularized_spectral_clustering = RegularizedSpectralClustering(n_clusters=n_clusters, must_link=must_link, cannot_link=cannot_link)
          regularized_spectral_clustering.fit(similarity_matrix_spectral, epochs=300, lr=0.01)
          y_pred_reg_spectral = regularized_spectral_clustering.predict()
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_reg_spectral)
          results[labeled_percent]['RegularizedSpectralClustering'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  Спектральная (Регуляризация): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")

          # в) Спектральная кластеризация с модифицированной матрицей схожести
          modified_similarity_spectral_clustering = ModifiedSimilaritySpectralClustering(n_clusters=n_clusters, must_link=must_link, cannot_link=cannot_link, random_state=random_state)
          y_pred_modified_spectral = modified_similarity_spectral_clustering.fit_predict(X_train)
          ari, nmi, fm = evaluate_clustering(y_train, y_pred_modified_spectral)
          results[labeled_percent]['ModifiedSimilaritySpectralClustering'] = {'ARI': ari, 'NMI': nmi, 'FM': fm}
          print(f"  Спектральная (Модиф. схожесть): ARI={ari:.3f}, NMI={nmi:.3f}, FM={fm:.3f}")
    return results


n_data = [100, 600, 2500, 6000]
#n_data = [1500]
#n_d=1500
for n_d in n_data:
  data, labels = load_caltech101(data_path='/content/', n_samples=n_d)
  torch.manual_seed(42)
  X, y = data, labels
  n_clusters = len(np.unique(y))
  le = LabelEncoder()
  y = le.fit_transform(y)
  results, y_pred_modified_spectral = compare_algorithms(X, y, n_clusters)